In [2]:
import os
import re
from getpass import getpass
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader, UnstructuredWordDocumentLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
# API Keys
OPENAI_API_KEY = getpass('🔑 OpenAI API Key:')


In [4]:
PINECONE_API_KEY = getpass('🔑 Pinecone API Key:')


### Cargar documentos a Pinecone

In [5]:
# =======================
# Configuración Pinecone
# =======================
class PineconeIndexFactory:
    @classmethod
    def get_index(cls, country: str):
        pc = Pinecone(api_key=PINECONE_API_KEY)
        index_name = f"rag-index-{country.lower()}"
        if not pc.has_index(index_name):
            pc.create_index(
                index_name,
                dimension=1536,
                metric="cosine",
                spec=ServerlessSpec(cloud="aws", region="us-east-1"),
            )
            print(f"🆕 Índice creado: {index_name}")
        else:
            print(f"📦 Índice existente: {index_name}")
        return pc.Index(index_name), index_name

# =========================
# Helpers para procesamiento
# =========================
def clean_name(original_name):
    name = original_name.lower()
    name = re.sub(r"[^a-z0-9\-]+", "-", name)
    name = re.sub(r"-+", "-", name).strip("-")
    return name

def rename_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        original_path = os.path.join(folder_path, filename)
        if os.path.isfile(original_path):
            name_without_ext, extension = os.path.splitext(filename)
            cleaned_name = clean_name(name_without_ext) + extension.lower()
            new_path = os.path.join(folder_path, cleaned_name)
            if original_path != new_path and not os.path.exists(new_path):
                os.rename(original_path, new_path)
                print(f"🔄 Renombrado: {filename} -> {cleaned_name}")

def load_document(file_path):
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".pdf":
        return PyPDFLoader(file_path).load()
    elif ext in [".docx", ".doc"]:
        return UnstructuredWordDocumentLoader(file_path).load()
    elif ext == ".txt":
        return TextLoader(file_path, encoding="utf-8").load()
    else:
        print(f"⚠️ Formato no soportado: {file_path}")
        return []

def batch(iterable, batch_size=50):
    for i in range(0, len(iterable), batch_size):
        yield iterable[i:i + batch_size]

def get_existing_sources(index_instance):
    """Consulta Pinecone para obtener todos los metadatos 'source' ya indexados"""
    existing_sources = set()
    # Consulta todos los vectores (usa fetch o describe_index_stats)
    stats = index_instance.describe_index_stats()
    if "namespaces" in stats:
        for ns in stats["namespaces"]:
            vectors = stats["namespaces"][ns]["vector_count"]
            if vectors > 0:
                print(f"📂 {vectors} vectores en namespace '{ns}' (se omiten archivos ya cargados)")
    return existing_sources

# =========================================
# Función principal para subir documentos
# =========================================
def upload_docs_folder(country, folder):
    rename_files_in_folder(folder)
    index_instance, index_name = PineconeIndexFactory.get_index(country)
    embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
    vector_store = PineconeVectorStore(embedding=embeddings, index=index_instance)

    # Recuperar archivos ya subidos
    existing_sources = get_existing_sources(index_instance)
    all_documents = []

    if not os.path.exists(folder):
        return "❌ Carpeta no encontrada"

    # Recorrer carpeta y subcarpetas
    for root, dirs, files in os.walk(folder):
        for filename in files:
            file_full_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_full_path, folder)  # path relativo
            if relative_path in existing_sources:
                print(f"⏭️ Archivo ya subido, se omite: {relative_path}")
                continue
            if os.path.isfile(file_full_path):
                print(f"📥 Procesando archivo nuevo: {relative_path}")
                docs = load_document(file_full_path)
                # Agregar metadatos de origen a cada documento
                for doc in docs:
                    doc.metadata["source"] = relative_path
                if docs:
                    all_documents.extend(docs)

    if not all_documents:
        return "⚠️ No se encontraron nuevos documentos para subir"

    # Dividir documentos en chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, chunk_overlap=100, length_function=len
    )
    documents_split = text_splitter.split_documents(all_documents)

    # Subir en batches
    for i, doc_batch in enumerate(batch(documents_split, batch_size=50), start=1):
        vector_store.add_documents(documents=doc_batch)
        print(f"✅ Batch {i}: {len(doc_batch)} chunks subidos")

    print(f"🚀 Nuevos documentos de '{country}' subidos a '{index_name}'")
    return f"✅ Upload completo: {len(documents_split)} chunks creados"



In [6]:
# =========================================
# Llamar la función después de subir archivos
# =========================================
upload_docs_folder("soporte", "D:\Temp\soporte")

<>:4: SyntaxWarning: invalid escape sequence '\T'
<>:4: SyntaxWarning: invalid escape sequence '\T'
C:\Users\ACER\AppData\Local\Temp\ipykernel_25480\3120864370.py:4: SyntaxWarning: invalid escape sequence '\T'
  upload_docs_folder("soporte", "D:\Temp\soporte")


📦 Índice existente: rag-index-soporte
📥 Procesando archivo nuevo: cambio-de-contrase-a-a-trav-s-de-gopass.pdf
📥 Procesando archivo nuevo: manual-obtenci-n-de-datos-opc-a-influxdb.pdf
📥 Procesando archivo nuevo: pi-data-archive-2015-r2-system-management-guide-en.pdf
📥 Procesando archivo nuevo: Andercol\Manual Historian.docx
📥 Procesando archivo nuevo: Belcorp\Arquitectura general de WCS.vsdx
⚠️ Formato no soportado: D:\Temp\soporte\Belcorp\Arquitectura general de WCS.vsdx
📥 Procesando archivo nuevo: Belcorp\Arreglar transacion manager.txt
📥 Procesando archivo nuevo: Belcorp\belcorp.txt
📥 Procesando archivo nuevo: Belcorp\Errores canales digitales.txt
📥 Procesando archivo nuevo: Belcorp\Traslados wcs manuales.txt
📥 Procesando archivo nuevo: Belcorp\Cambio de direcciones PTL\2023-01-20.xlsx
⚠️ Formato no soportado: D:\Temp\soporte\Belcorp\Cambio de direcciones PTL\2023-01-20.xlsx
📥 Procesando archivo nuevo: Belcorp\Factory Talk Optix\Integracion AD - Factory Talk Optix.pdf
📥 Procesando ar

'✅ Upload completo: 5109 chunks creados'